# ***DISTRIBUCIÓN DE SEGMENTOS***

### DataFrames:

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa_q" in nombre and "puebla" in nombre
    #if nombre.startswith("df_") in nombre and "puebla" in nombre
    #if nombre.startswith("df_") and "alfa_q" and "puebla" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_q_puebla']

segment_df_feb_2024_puebla = df_feb_2024_puebla[['Category']]
segment_df_jul_2023_puebla = df_jul_2023_puebla[['Category']]
segment_df_mar_2024_puebla = df_mar_2024_puebla[['Category']]
segment_df_may_2024_puebla = df_may_2024_puebla[['Category']]
segment_df_sep_2023_puebla = df_sep_2023_puebla[['Category']]
#df_alfa_q_puebla

dataframes_list = [
    segment_df_jul_2023_puebla,
    segment_df_feb_2024_puebla,   
]

In [2]:
df_alfa_q_puebla.rename(columns={'id_jul_23': 'id','Category':'categoria'}, inplace=True)

In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'categoria']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

### CONTEO ÚNICO DE CATEGORÍAS

In [4]:
conteo_total = {}
for df in dataframes_list:
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,S,68
1,A1,19
2,B2,16
3,B1,15
4,A3,13
5,C3,12
6,A2,11
7,C2,9
8,B3,8
9,C1,8


In [5]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,68,S,1
1,19,A,1
2,16,B,2
3,15,B,1
4,13,A,3
5,12,C,3
6,11,A,2
7,9,C,2
8,8,B,3
9,8,C,1


In [6]:
#segmentos_dist = conteo_total_df.groupby(['segmento', 'clase']) # Agrupar
segmentos_dist = conteo_total_dfs.groupby(['segmento', 'clase'])['conteo_total'].sum()
segmentos_dist

segmento  clase
A         1        19
          2        11
          3        13
B         1        15
          2        16
          3         8
C         1         8
          2         9
          3        12
D         1         6
          2         1
E         3         0
S         1        68
Name: conteo_total, dtype: int64

## Porcentajes totales de registros por segmento

In [7]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  186


,segmento,total_por_segmento,porcentaje_del_total
0,A,43,23.118280
1,B,39,20.967742
2,C,29,15.591398
3,D,7,3.763441
4,E,0,0.000000
5,S,68,36.559140


In [8]:
#Obtener el total de registros por segmento y clase
total_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum()
#Calcular los porcentajes
segmentos_percent = conteo_total_df.merge(total_segmento, on='segmento', suffixes=('', '_total'))
segmentos_percent['porcentaje'] = segmentos_percent['conteo_total'] / segmentos_percent['conteo_total_total'] * 100
#Seleccionar las columnas necesarias y eliminar las columnas auxiliares
segmentos_percent = segmentos_percent[['segmento', 'clase', 'porcentaje']]
segmentos_percent

,segmento,clase,porcentaje
0,S,1,100.000000
1,A,1,44.186047
2,B,2,41.025641
3,B,1,38.461538
4,A,3,30.232558
5,C,3,41.379310
6,A,2,25.581395
7,C,2,31.034483
8,B,3,20.512821
9,C,1,27.586207


Porcentaje de cada clase en relación al % que representa su segmento del total.

In [9]:
temp_dfs = [] # Almacenar los DataFrames temporales de cada segmento
# Realizamos un bucle sobre cada segmento
for segmento in resultados['segmento'].unique():
    # Filtramos las clases correspondientes al segmento actual
    clases_segmento = segmentos_percent[segmentos_percent['segmento'] == segmento]
    
    # Calculamos el total de porcentaje del segmento actual
    total_porcentaje_segmento = resultados[resultados['segmento'] == segmento]['porcentaje_del_total'].iloc[0]
    
    # Calculamos el porcentaje que representa cada clase en relación al porcentaje del segmento
    porcentaje_clase = (clases_segmento['porcentaje'] / 100) * total_porcentaje_segmento
    
    # DataFrame temporal para el segmento actual
    temp_df = pd.DataFrame({
        'segmento': [segmento] * len(clases_segmento),
        'clase': clases_segmento['clase'].values,
        'porcentaje_clase': porcentaje_clase#.round()
    })
    
    # Agrega DataFrame temporal a la lista
    temp_dfs.append(temp_df)

# Concatena todos los DataFrames temporales en un solo DataFrame
porcentajes_clase = pd.concat(temp_dfs, ignore_index=True)
porcentajes_clase

,segmento,clase,porcentaje_clase
0,A,1,10.215054
1,A,3,6.989247
2,A,2,5.913978
3,B,2,8.602151
4,B,1,8.064516
5,B,3,4.301075
6,C,3,6.451613
7,C,2,4.838710
8,C,1,4.301075
9,D,1,3.225806


In [10]:
porcentajes_clase_ordenado = porcentajes_clase.pivot(index='segmento', columns='clase', values='porcentaje_clase')
porcentajes_clase_ordenado.fillna(0, inplace=True)
porcentajes_clase_ordenado

clase,1,2,3
segmento,,,
A,10.215054,5.913978,6.989247
B,8.064516,8.602151,4.301075
C,4.301075,4.838710,6.451613
D,3.225806,0.537634,0.000000
S,36.559140,0.000000,0.000000


## Gráfica

In [11]:
segmentos = list(porcentajes_clase_ordenado.index)
segmentos_sorted = ['E','D','C','B','A','S']  # Orden deseado
clases = porcentajes_clase_ordenado.columns # Clases como etiquetas de las series de datos
clases = ['1', '2', '3']  # Clases en el orden deseado
# Asegúrate de que todos los segmentos mencionados estén en los datos con valor 0 si no existen
for segmento in segmentos_sorted:
    if segmento not in porcentajes_clase_ordenado.index:
        porcentajes_clase_ordenado.loc[segmento] = [0] * len(clases)

# Reordenar el DataFrame para seguir el orden deseado en `segmentos_sorted`
porcentajes_clase_ordenado = porcentajes_clase_ordenado.loc[segmentos_sorted]
colores = {
    '1': '#2962ff',
    '2': '#9500ff',
    '3': '#ff0059'
}
# Almacenar las barras de cada clase
barras = []
for clase in clases:
    barras.append(go.Bar(
        y=segmentos_sorted,  # Eje y: Segmentos
        x=porcentajes_clase_ordenado[clase],  # Eje x: Porcentajes de clase
        name=f'{clase}',  # Nombre de la serie de datos
        orientation='h',  # Orientación horizontal
        text=porcentajes_clase_ordenado[clase].apply(lambda x: f'{x:.2f}%'),  # Texto con los porcentajes
        textposition='auto',  # Posición automática del texto
        marker_color=colores[clase]   # Color de las barras
    ))

fig = go.Figure(data=barras)
fig.update_layout(
    yaxis=dict(
        title='Segmento', # Título del eje y        
    ),
    legend_title="Clase",
    barmode='stack',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico      
    xaxis=dict(
        title='Porcentaje', # Título del eje x
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
    ),  
    margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)
)

# Función para guardar la gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_dist_segmentos', carpeta='assets\graficas')

fig.show()